In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("spotify_millsongdata.csv")

In [20]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [21]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [22]:
df.shape

(57650, 4)

In [23]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [24]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [25]:
df.head(10)

,artist,song,text
0,Barbra Streisand,Everything Must Change,Everything must change \r\nNothing stays the ...
1,Bob Seger,The Aftermath,Dressed up to the nines \r\nDancin' three-qua...
2,Cyndi Lauper,Lorraine,Long ago a lonely man stood \r\nOff of a quie...
3,Moody Blues,Painted Smile,I can sing \r\nI can dance \r\nJust give me ...
4,Nickelback,Someday,How the hell'd we wind up like this? \r\nWhy ...
5,The Killers,Four Winds,"Your class, your caste, your country, sect, yo..."
6,Bonnie Raitt,Something To Talk About,People are talkin' 'bout people \r\nI hear th...
7,Bee Gees,Claustrophobia,I'd like to be the one to see your loving mind...
8,Phil Collins,Testify,I wanna testify \r\nTestify my love for you ...
9,Status Quo,Good Sign,"Nice to see \r\nNice to see you, I've been fe..."


In [26]:
df['text'][0]

"Everything must change  \r\nNothing stays the same  \r\nEveryone must change  \r\nNo one stays the same  \r\nThe young become the old  \r\nAnd mysteries do unfold  \r\nCause that's the way of time  \r\nNothing and no one goes unchanged  \r\nThere are not many things in life  \r\nYou can be sure of  \r\nExcept rain comes from the clouds  \r\n  \r\nSun lights up the sky  \r\nAnd hummingbirds do fly  \r\nWinter turns to spring  \r\nA wounded heart will heal  \r\nBut never much too soon  \r\n  \r\nYes  \r\nEverything must change  \r\nThe young become the old  \r\nAnd mysteries do unfold  \r\nCause that's the way of time  \r\nNothing and no one goes unchanged  \r\nThere are not many things in life  \r\nYou can be sure of  \r\nExcept rain comes from the clouds  \r\n  \r\nSun lights up the sky  \r\nAnd butterflies do fly  \r\nRain comes from the clouds  \r\nSun lights up the sky  \r\nAnd music  \r\nAnd music  \r\nMakes me cry\r\n\r\n"

In [27]:
# df = df.sample(5000)

In [28]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [29]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [83]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [111]:
# df['text'] = df['text'].apply(lambda x: tokenization(x))

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [86]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [87]:
similarity[0]

array([1.        , 0.01956213, 0.05756609, ..., 0.01869481, 0.00925578,
       0.04353269])

In [106]:
df[df['song'] == 'Everything Must Change']

,artist,song,text
0,Barbra Streisand,Everything Must Change,everything must change \r nothing stays the s...
296,Quincy Jones,Everything Must Change,everything must change \r nothing stays the s...


In [107]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [108]:
print(df)

                artist                              song   
0     Barbra Streisand            Everything Must Change  \
1            Bob Seger                     The Aftermath   
2         Cyndi Lauper                          Lorraine   
3          Moody Blues                     Painted Smile   
4           Nickelback                           Someday   
...                ...                               ...   
4995           Genesis                 Jesus He Knows Me   
4996    Billie Holiday  Do Nothing Till You Hear From Me   
4997            Unseen         Weapons Of Mass Deception   
4998        Kanye West                            H.A.M.   
4999     Dream Theater                         Invisible   

                                                   text  
0     everything must change  \r nothing stays the s...  
1     dressed up to the nines  \r dancin' three-quar...  
2     long ago a lonely man stood  \r off of a quiet...  
3     i can sing  \r i can dance  \r just give 

In [109]:
recommendation('Everything Must Change')

['Everything Must Change',
 'Here Comes The Sun',
 'Everything That Comes Around',
 'All Of The Lights',
 'One Of The Mysteries Of Life',
 "I Can't Stop The Rain",
 'Queen Of Rain',
 'Melodies From Heaven',
 'The World Turned Upside Down',
 'Rain Rain Rain',
 'I Wish It Would Rain Down',
 'What Goes Around Comes Around',
 'Along Comes A Woman',
 'The Sun Is Burning',
 "I'm On The Up",
 'Sure Got Cold After The Rain Fell',
 'When I Change My Life',
 'Utopia',
 'Change Of Heart',
 'Music And Me']

In [110]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))